# JIRA Delay Analysis - October to November

**Goal**: Identify top 5 and top 10 themes causing delays from October to November using LLM-assisted sentiment analysis

**Approach**:
1. Load and clean JIRA data
2. Filter October issues delayed to November
3. Extract and analyze comments using GPT
4. Cluster themes and identify root causes
5. Generate actionable management insights

## 1. Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Add src to path for custom modules
sys.path.append('../src')

# Load environment variables
load_dotenv()

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Environment setup complete")

## 2. Load Data

In [ ]:
# Load JIRA CSV data
# Update the filename to match your actual JIRA export
data_path = '../data/raw/jira_export.csv'

# Check if file exists
if not os.path.exists(data_path):
    print(f"⚠️ File not found: {data_path}")
    print("Please place your JIRA CSV export in the data/raw/ directory")
else:
    df = pd.read_csv(data_path)
    print(f"✅ Loaded {len(df)} records")
    print(f"\nColumns: {list(df.columns)}")
    df.head()

In [ ]:
# Data quality check
print("Data Quality Summary:")
print("=" * 50)
print(f"Total records: {len(df)}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nData types:\n{df.dtypes}")

## 3. Data Preprocessing

In [ ]:
# Convert date columns to datetime
# Adjust column names based on your actual JIRA export
date_columns = ['Created', 'Updated', 'Resolved']  # Update these column names

for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print("✅ Date columns converted")

In [ ]:
# Filter for October issues delayed to November
# Define October and November date ranges
october_start = pd.Timestamp('2024-10-01')
october_end = pd.Timestamp('2024-10-31')
november_start = pd.Timestamp('2024-11-01')
november_end = pd.Timestamp('2024-11-30')

# Filter logic: Created in October, Resolved in November (or still open)
# Adjust column names to match your data
delayed_issues = df[
    (df['Created'] >= october_start) & 
    (df['Created'] <= october_end) &
    (
        (df['Resolved'] >= november_start) |
        (df['Resolved'].isnull())
    )
].copy()

print(f"✅ Found {len(delayed_issues)} delayed issues")
delayed_issues.head()

## 4. Comment Extraction and Preparation

In [ ]:
# Extract comments from delayed issues
# Adjust 'Comment' column name to match your data
comment_column = 'Comment'  # Or 'Comments', 'Description', etc.

if comment_column in delayed_issues.columns:
    # Remove null comments
    issues_with_comments = delayed_issues[delayed_issues[comment_column].notna()].copy()
    print(f"✅ {len(issues_with_comments)} issues have comments")
    
    # Sample of comments
    print("\nSample comments:")
    for idx, comment in issues_with_comments[comment_column].head(3).items():
        print(f"\n--- Issue {idx} ---")
        print(comment[:200] + "..." if len(str(comment)) > 200 else comment)
else:
    print(f"⚠️ Column '{comment_column}' not found. Available columns: {list(delayed_issues.columns)}")

## 5. LLM-Assisted Sentiment Analysis

In [ ]:
# Import custom LLM analyzer
from llm_analyzer import DelayThemeAnalyzer

# Initialize analyzer
analyzer = DelayThemeAnalyzer()

print("✅ LLM analyzer initialized")

In [ ]:
# Analyze comments for delay themes
# This will make API calls to OpenAI - monitor your usage

delay_themes = []

for idx, row in tqdm(issues_with_comments.iterrows(), total=len(issues_with_comments), desc="Analyzing comments"):
    issue_key = row.get('Key', idx)
    comment_text = row[comment_column]
    
    # Get delay theme from GPT
    theme_result = analyzer.extract_delay_theme(comment_text, issue_key)
    delay_themes.append(theme_result)

# Convert to DataFrame
themes_df = pd.DataFrame(delay_themes)
print(f"\n✅ Analyzed {len(themes_df)} issues")
themes_df.head()

## 6. Theme Clustering and Analysis

In [ ]:
# Count theme frequencies
theme_counts = themes_df['theme'].value_counts()

print("Top 10 Delay Themes:")
print("=" * 50)
for i, (theme, count) in enumerate(theme_counts.head(10).items(), 1):
    percentage = (count / len(themes_df)) * 100
    print(f"{i}. {theme}: {count} ({percentage:.1f}%)")

In [ ]:
# Visualize top 10 themes
plt.figure(figsize=(12, 6))
top_10_themes = theme_counts.head(10)
sns.barplot(x=top_10_themes.values, y=top_10_themes.index, palette='viridis')
plt.xlabel('Number of Issues', fontsize=12)
plt.ylabel('Delay Theme', fontsize=12)
plt.title('Top 10 Delay Themes - October to November', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/top_10_delay_themes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved visualization to reports/top_10_delay_themes.png")

## 7. Sentiment Distribution

In [ ]:
# Analyze sentiment distribution
if 'sentiment' in themes_df.columns:
    sentiment_counts = themes_df['sentiment'].value_counts()
    
    plt.figure(figsize=(8, 6))
    colors = {'negative': '#e74c3c', 'neutral': '#95a5a6', 'positive': '#2ecc71'}
    sentiment_counts.plot(kind='bar', color=[colors.get(x, '#3498db') for x in sentiment_counts.index])
    plt.xlabel('Sentiment', fontsize=12)
    plt.ylabel('Number of Issues', fontsize=12)
    plt.title('Sentiment Distribution in Delayed Issues', fontsize=14, fontweight='bold')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig('../reports/sentiment_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

## 8. Actionable Insights

In [ ]:
# Generate actionable recommendations
top_5_themes = theme_counts.head(5)
top_10_themes = theme_counts.head(10)

print("="*70)
print("ACTIONABLE MANAGEMENT INSIGHTS")
print("="*70)
print(f"\nAnalysis Period: October 2024 → November 2024")
print(f"Total Delayed Issues: {len(delayed_issues)}")
print(f"Issues with Comments: {len(issues_with_comments)}")
print(f"\n{'='*70}\n")

print("TOP 5 CRITICAL DELAY THEMES:")
print("-"*70)
for i, (theme, count) in enumerate(top_5_themes.items(), 1):
    percentage = (count / len(themes_df)) * 100
    print(f"\n{i}. {theme.upper()}")
    print(f"   Impact: {count} issues ({percentage:.1f}%)")
    print(f"   Recommendation: [Add specific action based on theme]")

print(f"\n{'='*70}\n")
print("FULL TOP 10 THEMES FOR COMPREHENSIVE PLANNING:")
print("-"*70)
for i, (theme, count) in enumerate(top_10_themes.items(), 1):
    percentage = (count / len(themes_df)) * 100
    print(f"{i}. {theme}: {count} issues ({percentage:.1f}%)")

## 9. Export Results

In [ ]:
# Export theme analysis to CSV
themes_df.to_csv('../data/processed/delay_themes_analysis.csv', index=False)
print("✅ Saved detailed analysis to data/processed/delay_themes_analysis.csv")

# Export top themes summary
top_themes_summary = pd.DataFrame({
    'Rank': range(1, 11),
    'Theme': top_10_themes.index,
    'Count': top_10_themes.values,
    'Percentage': (top_10_themes.values / len(themes_df) * 100).round(1)
})

top_themes_summary.to_csv('../reports/top_10_themes_summary.csv', index=False)
print("✅ Saved top 10 summary to reports/top_10_themes_summary.csv")

# Export top 5 for management
top_5_summary = top_themes_summary.head(5)
top_5_summary.to_csv('../reports/top_5_critical_themes.csv', index=False)
print("✅ Saved top 5 critical themes to reports/top_5_critical_themes.csv")

## 10. Next Steps

### Immediate Actions:
1. Review top 5 themes with team leads
2. Assign owners to each critical theme
3. Create improvement initiatives based on findings
4. Set KPIs to track improvement in next sprint

### Continuous Improvement:
- Run this analysis monthly to track trends
- Compare month-over-month theme changes
- Measure effectiveness of interventions
- Adjust team processes based on data